In [1]:
from feature_analysis3 import *

In [2]:
target_imageset = 'oasis'
model_name = 'resnet50_vicreg'
train_type = 'selfsupervised'
model_string = '_'.join([model_name, train_type])
model_option = {'model_name': model_name,
                'train_type': train_type}

In [4]:
#resnet50 = torch.hub.load('facebookresearch/vicreg:main', 'resnet50')

In [ ]:
from torchvision.datasets.utils import download_url
weights_url = 'https://dl.fbaipublicfiles.com/vicreg/resnet50_fullckpt.pth'
weights_path = 'custom/vicreg_repo/resnet50_fullckpt.pth'
if not os.path.exists(weights_path):
    download_url(weights_url, 'custom/vicreg_repo')

In [ ]:
#backbone, embedding = resnet.__dict__['resnet50'](zero_init_residual=True)
#state_dict = torch.load(weights_path, map_location="cpu")

In [13]:
import custom.vicreg_repo.resnet as resnet

def resnet50(pretrained=True, **kwargs):
    model = resnet.resnet50(**kwargs)
    if pretrained:
        state_dict = torch.hub.load_state_dict_from_url(
            url="https://dl.fbaipublicfiles.com/vicreg/resnet50.pth",
            map_location='cpu',
        )
        model[0].load_state_dict(state_dict, strict=True)
    return model[0]

In [14]:
model = resnet50()
model = model.eval()
if torch.cuda.is_available():
    model = model.cuda()

In [16]:
image_data = load_image_data(target_imageset)
response_data = load_response_data(target_imageset)

In [17]:
imagenet_stats = {'mean': [0.485, 0.456, 0.406], 
                  'std':  [0.229, 0.224, 0.225]}

image_transforms = transforms.Compose([
    transforms.Resize((224,224)), 
    transforms.ToTensor(),
    transforms.Normalize(**imagenet_stats)
])

In [ ]:
stimulus_loader = get_stimulus_loader(image_data.image_path, image_transforms)

In [ ]:
stimulus_features = get_all_feature_maps(model_string, inputs = stimulus_loader)
stimulus_features = get_feature_map_srps(stimulus_features, delete_originals = True)

In [ ]:
reg_results = get_regression_results(model_option, stimulus_features, response_data, alpha_values = [1000])

In [ ]:
max_transform(reg_results[reg_results['image_type'] == 'Combo'], group_vars = ['measurement', 'image_type'])